# Project Description : Next WOrd Prediction Using LSTM
### Project Description

This project aims to develop a deep learning model for predicting the next word in a given sequence of words.
The model is built using LSTM network, which are well-suited for sequence prediction tasks. THe project include  the following steps:

1.  Data Collection
    we use the text of shakespear's "Hamlet" as out dataset .The rich, Complex text provides a good challenge for our model.

2. Data Preprocessing : The text data is tokenized, Converted into sequence, and padded to ensure uniform input lengths. THe sequence are then split into training and testing sets.

3. Model Building : An LSTM model is constructed with an embedding layer, two LSTM layers and a dense output layer with a softmax activation function to predict the probability of the next word.

4. Model Training : The model is trained using the prepared sequence, with early stopping implemented to prevent overfitting. Early stopping monitors the  validation loss and stops training when the loss stops improving.

5. Model Evaluation : The Mmodel is evaluated using a set of example sentences to test its ability to predict the next word accurately.

6. Deployment : A streamlit web application is developed to allow users to input a sequence of words and get the predicted next word in real-time.

In [ ]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

In [ ]:
#Load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

## Save to a file 
with open('hamlet.txt','w') as file :
    file.write(data)


In [ ]:
# Data preprocessing
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# Load the dataset

with open('hamlet.txt','r') as file :
    text = file.read().lower()

## Tokenize the text-creating indexes for words

tokenizer = Tokenizer()

tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

In [ ]:
tokenizer.word_index

In [1]:
# Create Input Sequences
input_sequence = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequence.append(n_gram_sequence)

SyntaxError: incomplete input (617353399.py, line 3)

In [ ]:
input_sequence

In [ ]:
# find the maximum length
max_sequence_length = max([len(x) for x in input_sequence])
max_sequence_length

In [ ]:
# all the sequence of the input sequence should be in equall length , so add the padding to it

input_sequence = np.array(pad_sequences(input_sequence,maxlen = max_sequence_length,padding = 'pre'))
input_sequence

In [ ]:
#Creating predicitors and label
import tensorflow as tf
x,y = input_sequence[:,:-1], input_sequence[:,-1]

In [ ]:
x

In [ ]:
y

In [ ]:
# Convert into categories
y = tf.keras.utils.to_categorical(y,num_classes = total_words)
y

In [ ]:
# Split the data into training and testing  sets
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
## Train our LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


# Define the model
model = Sequential()
model.add(Embedding(total_words,100,input_length = max_sequence_length-1))
model.add(LSTM(150,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation = 'softmax'))

In [ ]:
## Complie the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
# Now train the model
history = model.fit(x_train,y_train,epochs = 50, validation_data = (x_test,y_test),verbose =1 )


In [ ]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):] # Ensure the sequence length matches max_length
    token_list = pad_sequences([token_list],maxlen = max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose =0)
    predict_word_index = np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predict_word_index:
            return word
    return None

In [ ]:
input_text = "TO be or not to be"
print(f"Input text: {input_text}")
max_sequence_len =model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer, input_text,max_sequence_len)
print(f"The predicted next word is {next_word}")

In [ ]:
#Save the model
model.save("next_word_lstm.h5")


## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle :
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)